In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import yfinance as yf


In [ ]:
# Load Apple stock data
df = yf.download("AAPL", start="2015-01-01", end="2023-12-31")
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df['Target'] = df['Close'].shift(-1)
df.dropna(inplace=True)
df.head()


In [ ]:
X = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
y = df['Target'].values


In [ ]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42, shuffle=False
)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:
class SimpleRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
def regression_accuracy(preds, targets, tolerance=0.05):
    percent_diff = torch.abs(preds - targets) / (targets + 1e-8)
    correct = (percent_diff < tolerance).float()
    return correct.mean().item()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleRegressor().to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss, train_acc = 0.0, 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        loss = loss_fn(preds, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        train_acc += regression_accuracy(preds, y_batch) * X_batch.size(0)

    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{num_epochs} - Train Loss: {train_loss:.4f} - Accuracy: {train_acc:.4f}")


In [ ]:
model.eval()
y_preds, y_actuals = [], []

with torch.inference_mode():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        y_preds.append(preds.cpu())
        y_actuals.append(y_batch.cpu())

y_preds = torch.cat(y_preds).numpy()
y_actuals = torch.cat(y_actuals).numpy()

y_preds_rescaled = scaler_y.inverse_transform(y_preds)
y_actuals_rescaled = scaler_y.inverse_transform(y_actuals)

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.plot(y_actuals_rescaled, label='Actual')
plt.plot(y_preds_rescaled, label='Predicted')
plt.title("Predicted vs Actual Stock Prices")
plt.xlabel("Time Step")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()
